In [3]:
"""
Energy Dashboard - corrected & robust version
- Fixes __init__ typos, safe concat, and column/date handling.
- Saves chart as dashboard_modular.png and exports CSVs/text.
"""

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


# -------------------------
# MODULE 1: CLASSES (OOP)
# -------------------------
class MeterReading:
    def __init__(self, date, kwh):
        self.date = pd.to_datetime(date)
        self.kwh = float(kwh)


class Building:
    def __init__(self, name: str):
        self.name = name
        self.readings = []  # list of MeterReading

    def add_reading(self, date, kwh):
        new_reading = MeterReading(date, kwh)
        self.readings.append(new_reading)

    def total_kwh(self):
        return sum(r.kwh for r in self.readings)

    def __str__(self):
        return f"{self.name} ({len(self.readings)} readings)"


# -------------------------
# MODULE 2: DATA INGESTION
# -------------------------
def load_data(file_list):
    print("...Loading Data...")
    all_frames = []

    for filename in file_list:
        p = Path(filename)
        if not p.exists():
            print(f"Warning: file not found: {filename} (skipping)")
            continue
        try:
            df = pd.read_csv(p)
            # Normalize column names
            df.columns = [c.strip() for c in df.columns]
            # Derive building name from filename if not present
            name = p.stem.replace('_', ' ').title()
            df['Building'] = df.get('Building', name)
            print(f"Read success: {filename}")
            all_frames.append(df)
        except Exception as e:
            print(f"Could not read {filename}: {e}")

    if not all_frames:
        print("No files were successfully read.")
        return pd.DataFrame()  # empty

    # Combine safely
    combined_df = pd.concat(all_frames, ignore_index=True, sort=False)

    # Ensure Date column exists and parse it
    if 'Date' not in combined_df.columns:
        raise KeyError("Required column 'Date' not found in input files.")
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
    combined_df = combined_df.dropna(subset=['Date'])

    # Ensure KWh column exists (allow 'KWh' or 'kwh' or 'kW h' variants)
    kwh_col = None
    for candidate in ['KWh', 'kwh', 'kW h', 'kW_h', 'kwh_total']:
        if candidate in combined_df.columns:
            kwh_col = candidate
            break
    # fallback: try to find any numeric-like column other than Date/Building
    if kwh_col is None:
        for c in combined_df.columns:
            if c not in ['Date', 'Building'] and pd.api.types.is_numeric_dtype(combined_df[c]):
                kwh_col = c
                break
    if kwh_col is None:
        raise KeyError("Could not find a numeric energy column (expected 'KWh' or similar).")
    # unify to 'KWh'
    combined_df['KWh'] = pd.to_numeric(combined_df[kwh_col], errors='coerce')
    combined_df = combined_df.dropna(subset=['KWh'])

    # Keep only relevant columns and sort by date
    combined_df = combined_df[['Date', 'Building', 'KWh']].sort_values('Date').reset_index(drop=True)
    return combined_df


# -------------------------
# MODULE 3: ANALYSIS
# -------------------------
def calculate_stats(df):
    print("...Calculating Stats...")
    daily_totals = df.groupby('Date')['KWh'].sum()
    b_summary = df.groupby('Building')['KWh'].agg(['mean', 'max', 'sum'])
    return daily_totals, b_summary


# -------------------------
# MODULE 4: VISUALIZATION
# -------------------------
def create_charts(df, daily_data, building_summary):
    print("...Drawing Charts...")
    # Prepare figure with 3 rows
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 14))

    # 1. Trend Line
    ax1.plot(daily_data.index, daily_data.values, marker='o', linestyle='-')
    ax1.set_title('Total Daily Energy Usage')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('KWh')
    ax1.grid(True)
    fig.autofmt_xdate(rotation=25)

    # 2. Bar Chart - average usage by building
    names = building_summary.index.astype(str)
    averages = building_summary['mean']
    ax2.bar(names, averages)
    ax2.set_title('Average Usage by Building')
    ax2.set_ylabel('Mean KWh')
    ax2.set_xticklabels(names, rotation=45, ha='right')

    # 3. Scatter Plot - Peak events per building
    for name in df['Building'].unique():
        subset = df[df['Building'] == name]
        ax3.scatter(subset['Date'], subset['KWh'], label=name, alpha=0.7)
    ax3.set_title('Peak Usage Events')
    ax3.set_xlabel('Date')
    ax3.set_ylabel('KWh')
    ax3.legend()
    ax3.grid(True)

    plt.tight_layout()
    outname = 'dashboard_modular.png'
    plt.savefig(outname)
    plt.close(fig)
    print(f"Chart saved as '{outname}'")


# -------------------------
# MODULE 5: REPORTING
# -------------------------
def generate_report(df, building_summary):
    print("...Generating Report...")
    total_energy = df['KWh'].sum()
    if not building_summary.empty:
        top_building = building_summary['sum'].idxmax()
    else:
        top_building = "N/A"

    summary_text = (
        "FINAL EXECUTIVE SUMMARY\n"
        "=======================\n"
        f"Total Energy Consumed: {total_energy:.2f} KWh\n"
        f"Highest Consuming Building: {top_building}\n"
    )

    with open('summary.txt', 'w') as f:
        f.write(summary_text)

    df.to_csv('cleaned_energy_data.csv', index=False)
    building_summary.to_csv('building_summary.csv')
    print(summary_text)


# -------------------------
# MAIN EXECUTION
# -------------------------
if __name__ == "__main__":
    # 1. Define our files (example list)
    my_files = ['library_block.csv', 'science_block.csv']

    try:
        final_df = load_data(my_files)
        if final_df.empty:
            print("Something went wrong, no data loaded.")
        else:
            daily_stats, building_stats = calculate_stats(final_df)
            create_charts(final_df, daily_stats, building_stats)
            generate_report(final_df, building_stats)

            # OOP Check (demonstration)
            lib = Building("Library")
            # add a reading just to show usage
            lib.add_reading(final_df.iloc[0]['Date'], final_df.iloc[0]['KWh'])
            print("OOP System Ready:", lib)
    except Exception as exc:
        print("An error occurred during execution:", exc)


...Loading Data...
No files were successfully read.
Something went wrong, no data loaded.
